In [ ]:
# Importar librerías necesarias
import sys
import os
from datetime import date, timedelta
import json

# Agregar el directorio Backend al path
sys.path.insert(0, os.path.abspath('.'))

print(f"Directorio actual: {os.getcwd()}")
print(f"Archivos en directorio: {os.listdir('.')}")

: 

In [ ]:
# Importar módulos del proyecto
from model.methods import predict_stock_product_date
from db.models import listar_productos
from db.functions import generate_csv
from model.retrain import retrain_from_csv

print(" Módulos importados correctamente")

2025-12-10 15:03:04.679088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-10 15:03:05.102164: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-10 15:03:05.174134: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-12-10 15:03:05.174181: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

## 1. Test: /predict/product-date

Predice el stock de un producto específico en una fecha específica.

In [ ]:
# Test inicial: Predicción de producto en fecha específica
print("=" * 60)
print("TEST: Predicción de producto en fecha específica")
print("=" * 60)

# Obtener un producto de ejemplo
productos = listar_productos()
producto_ejemplo = productos[0] if productos else "PROD-001"
fecha_ejemplo = (date.today() + timedelta(days=7)).strftime("%Y-%m-%d")

print(f"\nProducto: {producto_ejemplo}")
print(f"Fecha: {fecha_ejemplo}")

resultado = predict_stock_product_date(
    product_id=producto_ejemplo,
    date=fecha_ejemplo
)

print(f"\n Resultado:")
print(json.dumps(resultado, indent=2, default=str))

## 2. Test: /predict/product

Predice el stock de un producto hasta que se agote (máximo 30 días).

In [ ]:
# Test inicial: Predicción de producto hasta que se agote
print("=" * 60)
print("TEST: Predicción de producto hasta agotamiento")
print("=" * 60)

producto_ejemplo = productos[0] if productos else "PROD-001"
print(f"\nProducto: {producto_ejemplo}")

day = date.today()
result = []

for i in range(30):
    day = day + timedelta(days=1)
    pred = predict_stock_product_date(
        product_id=producto_ejemplo,
        date=day
    )
    
    result.append({
        "product_name": producto_ejemplo,
        "predicted_stock": int(pred["predicted_stock"]),
        "date": day.strftime("%Y-%m-%d")
    })
    
    if pred["predicted_stock"] <= 0:
        print(f"  Producto se agotará el: {day}")
        break

print(f"\n Resultados ({len(result)} días predichos):")
for r in result[:5]:  # Mostrar solo los primeros 5
    print(f"  - {r['date']}: Stock = {r['predicted_stock']}")
if len(result) > 5:
    print(f"  ... ({len(result)-5} días más)")

## 3. Test: /predict/date

Predice el stock de todos los productos en una fecha específica.

In [ ]:
# Test inicial: Predicción de todos los productos en fecha específica
print("=" * 60)
print("TEST: Predicción de todos los productos en fecha específica")
print("=" * 60)

fecha_ejemplo = (date.today() + timedelta(days=15)).strftime("%Y-%m-%d")
print(f"\nFecha: {fecha_ejemplo}")
print(f"Total de productos: {len(productos)}")

results = []
for product in productos:
    predi = predict_stock_product_date(
        product_id=product,
        date=fecha_ejemplo
    )
    results.append({
        "product_name": product,
        "predicted_stock": int(predi["predicted_stock"]),
        "date": fecha_ejemplo
    })

print(f"\n Resultados (mostrando primeros 5):")
for r in results[:5]:
    print(f"  - {r['product_name']}: Stock = {r['predicted_stock']}")
if len(results) > 5:
    print(f"  ... ({len(results)-5} productos más)")

# Productos con bajo stock
bajo_stock = [r for r in results if r['predicted_stock'] <= 10]
print(f"\n  Productos con stock <= 10: {len(bajo_stock)}")

## 4. Test: /predict/all

Predice el stock de todos los productos hasta que alguno se agote.

In [ ]:
# Test inicial: Predicción de todos los productos hasta agotamiento
print("=" * 60)
print("TEST: Predicción de todos los productos hasta agotamiento")
print("=" * 60)

day = date.today()
results = []

for i in range(30):
    zero = False
    for product in productos:
        pred = predict_stock_product_date(
            product_id=product,
            date=day
        )
        
        results.append({
            "product_name": product,
            "predicted_stock": int(pred["predicted_stock"]),
            "current_stock": pred["current_stock"],
            "date": day.strftime("%Y-%m-%d")
        })
        
        if int(pred["predicted_stock"]) <= 0:
            zero = True
    
    day = day + timedelta(days=1)
    
    if zero:
        print(f"  Al menos un producto se agotará alrededor del: {day}")
        break

print(f"\n Total de predicciones generadas: {len(results)}")
print(f"Días simulados: {i+1}")

# Mostrar últimas predicciones de cada producto
print("\nÚltimas predicciones por producto (primeros 3):")
for product in productos[:3]:
    prod_results = [r for r in results if r['product_name'] == product]
    if prod_results:
        ultimo = prod_results[-1]
        print(f"  - {ultimo['product_name']}: Stock final = {ultimo['predicted_stock']}")

## 5. Test: Generar Reporte CSV

Genera un reporte en formato CSV.

In [ ]:
# Test: Generar reporte CSV
print("=" * 60)
print("TEST: Generar Reporte CSV")
print("=" * 60)

# Generar reporte del mes actual
mes_actual = date.today().month
print(f"\nGenerando reporte para el mes: {mes_actual}")

try:
    file_path = generate_csv(mes_actual)
    
    if file_path and os.path.exists(file_path):
        print(f" Archivo CSV generado: {file_path}")
        print(f"Tamaño: {os.path.getsize(file_path)} bytes")
        
        # Leer primeras líneas
        with open(file_path, 'r') as f:
            lines = f.readlines()[:5]
            print("\nPrimeras líneas del CSV:")
            for line in lines:
                print(f"  {line.strip()}")
    else:
        print(" No se pudo generar el archivo CSV")
except Exception as e:
    print(f" Error: {str(e)}")

## 6. Reentrenamiento del Modelo

Reentrena el modelo con nuevos datos desde un archivo CSV.

In [ ]:
# Reentrenamiento con CSV
print("=" * 60)
print("REENTRENAMIENTO DEL MODELO")
print("=" * 60)

# Usar el CSV existente de datos
csv_path = "files/data20oct-2nov.csv"

if os.path.exists(csv_path):
    print(f"\nUsando CSV: {csv_path}")
    print(f"Tamaño: {os.path.getsize(csv_path)} bytes")
    
    # Leer contenido del CSV
    with open(csv_path, 'rb') as f:
        csv_content = f.read()
    
    print("\n⏳ Iniciando reentrenamiento...")
    print("Esto puede tomar varios minutos...")
    
    try:
        resultado = retrain_from_csv(
            csv_content=csv_content,
            filename="data20oct-2nov.csv",
            epochs=10,
            batch_size=128,
            umbral_degradacion=0.1
        )
        
        print("\n Resultado del reentrenamiento:")
        print(json.dumps(resultado, indent=2, default=str))
        
        if resultado.get("success"):
            print("\n Modelo reentrenado exitosamente")
        else:
            print("\n  Reentrenamiento completado con advertencias")
    except Exception as e:
        print(f"\n Error durante el reentrenamiento: {str(e)}")
else:
    print(f" No se encontró el archivo CSV en: {csv_path}")

## 7. Test Post-Reentrenamiento: /predict/product

Prueba la predicción después del reentrenamiento para comparar resultados.

In [ ]:
# Test POST-reentrenamiento: Predicción de producto
print("=" * 60)
print("TEST POST-REENTRENAMIENTO: Predicción de producto")
print("=" * 60)

producto_ejemplo = productos[0] if productos else "PROD-001"
print(f"\nProducto: {producto_ejemplo}")

day = date.today()
result_post = []

for i in range(30):
    day = day + timedelta(days=1)
    pred = predict_stock_product_date(
        product_id=producto_ejemplo,
        date=day
    )
    
    result_post.append({
        "product_name": producto_ejemplo,
        "predicted_stock": int(pred["predicted_stock"]),
        "date": day.strftime("%Y-%m-%d")
    })
    
    if pred["predicted_stock"] <= 0:
        print(f"  Producto se agotará el: {day}")
        break

print(f"\n Resultados POST-reentrenamiento ({len(result_post)} días):")
for r in result_post[:5]:
    print(f"  - {r['date']}: Stock = {r['predicted_stock']}")
if len(result_post) > 5:
    print(f"  ... ({len(result_post)-5} días más)")

print("\n Comparación con resultados anteriores disponible arriba")

## 8. Test Post-Reentrenamiento: /predict/product-date

Prueba la predicción de producto en fecha específica después del reentrenamiento.

In [ ]:
# Test POST-reentrenamiento: Predicción con fecha específica
print("=" * 60)
print("TEST POST-REENTRENAMIENTO: Predicción producto + fecha")
print("=" * 60)

producto_ejemplo = productos[0] if productos else "PROD-001"
fecha_ejemplo = (date.today() + timedelta(days=7)).strftime("%Y-%m-%d")

print(f"\nProducto: {producto_ejemplo}")
print(f"Fecha: {fecha_ejemplo}")

resultado_post = predict_stock_product_date(
    product_id=producto_ejemplo,
    date=fecha_ejemplo
)

print(f"\n Resultado POST-reentrenamiento:")
print(json.dumps(resultado_post, indent=2, default=str))

## 9. Test Post-Reentrenamiento: /predict/date

Prueba la predicción de todos los productos en fecha específica después del reentrenamiento.

In [ ]:
# Test POST-reentrenamiento: Todos los productos en fecha
print("=" * 60)
print("TEST POST-REENTRENAMIENTO: Todos los productos en fecha")
print("=" * 60)

fecha_ejemplo = (date.today() + timedelta(days=15)).strftime("%Y-%m-%d")
print(f"\nFecha: {fecha_ejemplo}")

results_post = []
for product in productos:
    predi = predict_stock_product_date(
        product_id=product,
        date=fecha_ejemplo
    )
    results_post.append({
        "product_name": product,
        "predicted_stock": int(predi["predicted_stock"]),
        "date": fecha_ejemplo
    })

print(f"\n Resultados POST-reentrenamiento (primeros 5):")
for r in results_post[:5]:
    print(f"  - {r['product_name']}: Stock = {r['predicted_stock']}")

bajo_stock_post = [r for r in results_post if r['predicted_stock'] <= 10]
print(f"\n  Productos con stock <= 10: {len(bajo_stock_post)}")

## 10. Test Post-Reentrenamiento: /predict/all

Prueba la predicción de todos los productos hasta agotamiento después del reentrenamiento.

In [ ]:
# Test POST-reentrenamiento: Todos los productos hasta agotamiento
print("=" * 60)
print("TEST POST-REENTRENAMIENTO: Todos hasta agotamiento")
print("=" * 60)

day = date.today()
results_post_all = []

for i in range(30):
    zero = False
    for product in productos:
        pred = predict_stock_product_date(
            product_id=product,
            date=day
        )
        
        results_post_all.append({
            "product_name": product,
            "predicted_stock": int(pred["predicted_stock"]),
            "current_stock": pred["current_stock"],
            "date": day.strftime("%Y-%m-%d")
        })
        
        if int(pred["predicted_stock"]) <= 0:
            zero = True
    
    day = day + timedelta(days=1)
    
    if zero:
        print(f"  Al menos un producto se agotará alrededor del: {day}")
        break

print(f"\n Total de predicciones POST-reentrenamiento: {len(results_post_all)}")
print(f"Días simulados: {i+1}")

print("\nÚltimas predicciones POST-reentrenamiento (primeros 3):")
for product in productos[:3]:
    prod_results = [r for r in results_post_all if r['product_name'] == product]
    if prod_results:
        ultimo = prod_results[-1]
        print(f"  - {ultimo['product_name']}: Stock final = {ultimo['predicted_stock']}")

## 11. Resumen Final

Resumen de todos los tests realizados.

In [ ]:
# Resumen final
print("=" * 60)
print("RESUMEN DE PRUEBAS COMPLETADAS")
print("=" * 60)

tests_completados = [
    " /predict/product-date (PRE)",
    " /predict/product (PRE)",
    " /predict/date (PRE)",
    " /predict/all (PRE)",
    " Generar reporte CSV",
    " Reentrenamiento del modelo",
    " /predict/product (POST)",
    " /predict/product-date (POST)",
    " /predict/date (POST)",
    " /predict/all (POST)"
]

print("\nTests ejecutados:")
for test in tests_completados:
    print(f"  {test}")

print("\n" + "=" * 60)
print("TODAS LAS PRUEBAS COMPLETADAS")
print("=" * 60)